In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6


In [39]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objs as go

import pandas as pd
import numpy as np


In [6]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [7]:
df=pd.read_csv('covid_19_data.csv')
df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
15126,15127,04/14/2020,Wyoming,US,2020-04-14 23:41:11,282.0,1.0,0.0
15127,15128,04/14/2020,Xinjiang,Mainland China,2020-04-14 23:41:11,76.0,3.0,73.0
15128,15129,04/14/2020,Yukon,Canada,2020-04-14 23:41:11,8.0,0.0,0.0
15129,15130,04/14/2020,Yunnan,Mainland China,2020-04-14 23:41:11,184.0,2.0,175.0


In [19]:
df.shape

(15131, 8)

In [24]:
df.isna().sum()


SNo                   0
ObservationDate       0
Province/State     7467
Country/Region        0
Last Update           0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [9]:
df_ts=df.groupby(['ObservationDate']).sum()
df_ts

<ipython-input-9-44d100a3b5cf>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ts=df.groupby(['ObservationDate']).sum()


,SNo,Confirmed,Deaths,Recovered
ObservationDate,,,,
01/22/2020,741,555.0,17.0,28.0
01/23/2020,2829,653.0,18.0,30.0
01/24/2020,4305,941.0,26.0,36.0
01/25/2020,6490,1438.0,42.0,39.0
01/26/2020,9071,2118.0,56.0,52.0
...,...,...,...,...
04/10/2020,4394490,1691719.0,102525.0,376096.0
04/11/2020,4497531,1771514.0,108503.0,402110.0
04/12/2020,4586080,1846680.0,114090.0,421722.0


In [27]:
df_bd=df.loc[df['Country/Region']=='Bangladesh']

In [33]:
df1=df.loc[df.ObservationDate==max(df.ObservationDate)]

df2=df1.groupby('Country/Region').sum()

df3=df2.sort_values(by='Deaths',ascending=False)

df3=df3.head(20)
df3

<ipython-input-33-e52ffe3379c4>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df2=df1.groupby('Country/Region').sum()


,SNo,Confirmed,Deaths,Recovered
Country/Region,,,,
US,873752,607670.0,25832.0,47763.0
Italy,14892,162488.0,21067.0,37130.0
Spain,14965,172541.0,18056.0,67504.0
France,165536,131361.0,15748.0,29098.0
UK,165275,94845.0,12129.0,323.0
Iran,14888,74877.0,4683.0,48129.0
Belgium,14827,31119.0,4157.0,6868.0
Mainland China,466931,82249.0,3341.0,77753.0
Germany,14873,131359.0,3294.0,68200.0


# DASHBOARD

In [46]:
app=dash.Dash()

app.layout= html.Div([html.H1('COVID-19 Dashboard'),
                      dcc.Graph(
        id='line_bd',
        figure={
                'data':[
                {'x': df_bd.ObservationDate, 'y': df_bd.Confirmed,'type':'bar','name':'Confirmed_bd'},
                {'x': df_bd.ObservationDate, 'y': df_bd.Deaths,'type':'bar','name':'Deaths_bd', 'marker' : { "color" : 'rgb(255,0,0)'}},
                {'x': df_bd.ObservationDate, 'y': df_bd.Recovered,'type':'bar','name':'Recovered_bd', 'marker' : { "color" : 'rgb(0,128,0)'}}
                        ],
                'layout':go.Layout(title='Bangladesh Time Series Cases')
                }),
                      dcc.Graph(
        id='line',
        figure={
                'data':[
                {'x': df_ts.index, 'y': df_ts.Confirmed,'name':'Confirmed'},
                {'x': df_ts.index, 'y': df_ts.Deaths,'name':'Deaths', 'marker' : { "color" : 'rgb(255,0,0)'}},
                {'x': df_ts.index, 'y': df_ts.Recovered,'name':'Recovered', 'marker' : { "color" : 'rgb(0,128,0)'}}
                        ],
                'layout':go.Layout(title='Worldwide Time Series Cases')
                }
            ),

    dcc.Graph(
        id='confirmed',
        figure={
                'data':[
                {'x': df3.index, 'y': df3.Confirmed, 'type':'bar','name':'Confirmed'}
                        ],
                'layout':go.Layout(title='Confirmed Cases by Country')
                }
            ),

    dcc.Graph(
        id='death',
        figure={
                'data':[
                {'x': df3.index, 'y': df3.Deaths, 'type':'bar','name':'Deaths', 'marker' : { "color" : 'rgb(255,0,0)'}}
                        ],
                'layout':go.Layout(title='Death Cases by Country')
                }
            ),

    dcc.Graph(
        id='recovered',
        figure={
                'data':[
                {'x': df3.index, 'y': df3.Recovered, 'type':'bar','name':'Recovered', 'marker' : { "color" : 'rgb(0,128,0)'}}
                        ],
                'layout':go.Layout(title='Recovered Cases by Country')
                }
            )
])




In [47]:
if __name__=="__main__":
    app.run_server(port=4051)

<IPython.core.display.Javascript object>